<a href="https://colab.research.google.com/github/posheadig/Space-Junk-Constellations/blob/main/Copy_of_SpaceJunk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Compute TLE Trajectories

In [ ]:
# Install dependencies.
!pip install skyfield pandas

In [ ]:
from skyfield.api import Topos, load, utc
import json
import numpy as np
import pandas as pd 
import datetime



In [ ]:
# Here we load a bunch of satellites.
stations_url = 'http://www.celestrak.com/NORAD/elements/active.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')


[#################################] 100% active.txt


Loaded 3844 satellites


In [ ]:
# Here we iterate through the satellites to take a look ..
for s in satellites[:1]:
  print(s.name)  
  

CALSPHERE 1


In [ ]:
# Generate a bunch of times in the future to calculate positions at.

# Function reference is here
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html

# Generate one week's worth of times, hourly.
# times = pd.date_range('2020-10-01', periods=7*24, freq='1H')

# Generate one days's worth of times, every 10 minutes for 24 hours.
times = pd.date_range('2020-10-01', periods=1*6*24, freq='10min')
times


DatetimeIndex(['2020-10-01 00:00:00', '2020-10-01 00:10:00',
               '2020-10-01 00:20:00', '2020-10-01 00:30:00',
               '2020-10-01 00:40:00', '2020-10-01 00:50:00',
               '2020-10-01 01:00:00', '2020-10-01 01:10:00',
               '2020-10-01 01:20:00', '2020-10-01 01:30:00',
               ...
               '2020-10-01 22:20:00', '2020-10-01 22:30:00',
               '2020-10-01 22:40:00', '2020-10-01 22:50:00',
               '2020-10-01 23:00:00', '2020-10-01 23:10:00',
               '2020-10-01 23:20:00', '2020-10-01 23:30:00',
               '2020-10-01 23:40:00', '2020-10-01 23:50:00'],
              dtype='datetime64[ns]', length=144, freq='10T')

In [ ]:
# Iterate all satellites for all times ...
# https://rhodesmill.org/skyfield/earth-satellites.html

ts = load.timescale()

entries = []

# Iterates all satellites in the TLE that was loaded.
for satellite in satellites[:20]:
  entry = { }
  entry['name'] = satellite.name
  xyzts = []
  for time in times:
    # Convert my timestamp to skyfield data type.
    t = ts.utc(pd.to_datetime(time, utc=True))

    # Ask for position data at the given time.
    geocentric = satellite.at(t)

    # Place data into a data structure.
    xyzt = {}
    xyzt['x'] = geocentric.position.km[0] # Get x
    xyzt['y'] = geocentric.position.km[1] # Get y
    xyzt['z'] = geocentric.position.km[2] # Get z
    xyzt['t'] = t.utc_datetime().timestamp() # Convert the time to epoch milliseconds
    xyzts.append(xyzt)
  entry['xyzts'] = xyzts
  entries.append(entry)

In [ ]:
# This data.json file will be over to the left in the folder ... button thing.
with open('data.json', 'w') as outfile:
    json.dump(entries, outfile)
